In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [23]:
# Dataset, DataLoader 준비

x, y = make_classification(
    n_samples=3000,
    n_features=5,
    n_informative=2,
    n_redundant=0,
    n_clusters_per_class=1,
    random_state=42
)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    
train_dataset = CustomDataset(x_train, y_train)
test_dataset = CustomDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Training

class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        
        self.linear = nn.Linear(input_dim, 1)
        
    def forward(self, x):
        out = self.linear(x)
        
        out = torch.sigmoid(out)
        
        return out
    
model = LogisticRegression(input_dim=5)

criterion = nn.BCELoss()

optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 100

for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        loss = criterion(outputs, targets.unsqueeze(1))
        
        loss.backward()
        
        optimizer.step()
        
    if epoch % 10 == 0:
        print(f"Epoch [{epoch + 1} / {num_epochs}], Loss: {loss.item():.4f}")
        
# Evaluation
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

with torch.no_grad():
    correct = 0
    total = 0
    
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        outputs = model(inputs)
        
        predicted = (outputs.data > 0.5).int().squeeze()
        
        total += targets.size(0)
        
        correct += (predicted == targets).sum().item()
        
    print("Accuracy: %d%%" % (100 * correct / total))

Epoch [1 / 100], Loss: 0.6123
Epoch [11 / 100], Loss: 0.2687
Epoch [21 / 100], Loss: 0.3955
Epoch [31 / 100], Loss: 0.1871
Epoch [41 / 100], Loss: 0.2921
Epoch [51 / 100], Loss: 0.3146
Epoch [61 / 100], Loss: 0.2293
Epoch [71 / 100], Loss: 0.2437
Epoch [81 / 100], Loss: 0.7150
Epoch [91 / 100], Loss: 0.3385
Device: cpu
Accuracy: 91%
